In [ ]:
%pip install "pandas<3.0.0"
%pip install pypots benchpots pygrinder --upgrade
%pip install scikit-learn   

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Importa bibliotecas necessárias para manipulação de dados e pré-processamento
import numpy as np
from sklearn.preprocessing import StandardScaler
from pygrinder import mcar, calc_missing_rate
from benchpots.datasets import preprocess_physionet2012

# Carrega e pré-processa o conjunto de dados PhysioNet 2012
# subset='set-a' seleciona o subconjunto A do dataset
# rate=0.1 adiciona artificialmente 10% de valores faltantes aos dados para teste
data = preprocess_physionet2012(subset='set-a', rate=0.1)

# Separa os dados em conjuntos de treino, validação e teste
# Cada conjunto contém séries temporais com valores faltantes (originais + artificiais)
train_X, val_X, test_X = data["train_X"], data["val_X"], data["test_X"]

# Exibe as dimensões dos dados: (n_amostras, n_passos_temporais, n_características)
print(train_X.shape)
print(val_X.shape)  # O número de amostras varia entre treino e validação, mas passos e características são iguais

# Calcula e exibe a taxa de valores faltantes no conjunto de treino
print(f"Temos {calc_missing_rate(train_X):.1%} valores faltantes em train_X")

# Prepara o conjunto de treino: apenas as séries temporais incompletas são necessárias
train_set = {"X": train_X}

# Prepara o conjunto de validação: inclui tanto os dados incompletos quanto os originais completos
# X_ori serve como ground truth para avaliar o desempenho do modelo e selecionar o melhor checkpoint
val_set = {
    "X": val_X,
    "X_ori": data["val_X_ori"],
}

# Prepara o conjunto de teste: contém apenas as séries temporais incompletas
# O modelo irá imputar (preencher) os valores faltantes neste conjunto
test_set = {"X": test_X}

# Obtém os valores originais completos do conjunto de teste para avaliação final
test_X_ori = data["test_X_ori"]

# Cria uma máscara que identifica os valores que foram artificialmente removidos
# Esta máscara indica onde estão os valores que existem em X_ori mas foram removidos em test_X
# Usa XOR (^) para encontrar posições que são faltantes em test_X mas presentes em test_X_ori
indicating_mask = np.isnan(test_X) ^ np.isnan(test_X_ori)

# Importa o modelo BRITS (Bidirectional Recurrent Imputation for Time Series)
from pypots.imputation import BRITS
from pypots.nn.functional import calc_mae

# Configura e inicializa o modelo BRITS com os hiperparâmetros:
# - n_steps: número de passos temporais na série
# - n_features: número de características/variáveis em cada passo
# - rnn_hidden_size: tamanho da camada oculta da RNN bidirecional
# - epochs: número de épocas de treinamento
brits = BRITS(
    n_steps=train_X.shape[1],
    n_features=train_X.shape[2],
    rnn_hidden_size=256,
    epochs=100,
)

# Treina o modelo usando o conjunto de treino e valida com o conjunto de validação
# O modelo aprende a reconstruir valores faltantes através de RNNs bidirecionais
brits.fit(train_set, val_set)

# Usa o modelo treinado para imputar (preencher) todos os valores faltantes no conjunto de teste
# Isso inclui tanto os valores originalmente faltantes quanto os artificialmente removidos
imputation = brits.impute(test_set)

# Calcula o erro absoluto médio (MAE) comparando os valores imputados com os valores verdadeiros
# Avalia apenas os valores que foram artificialmente removidos (indicados pela máscara)
mae = calc_mae(imputation, np.nan_to_num(test_X_ori), indicating_mask)

print(f"MAE: {mae:.4f}")

# Salva o modelo treinado em arquivo para uso futuro
# overwrite=True permite sobrescrever o arquivo se ele já existir
brits.save("save_it_here/brits_physionet2012.pypots", overwrite=True)

# Recarrega o modelo salvo do arquivo
# Útil para continuar o treinamento ou realizar novas imputações posteriormente
brits.load("save_it_here/brits_physionet2012.pypots")


2026-02-10 18:30:23 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2026-02-10 18:30:23 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2026-02-10 18:30:23 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2026-02-10 18:30:23 [INFO]: Loaded successfully!
/home/luis/code/baselines_temporal_data_inputation/.venv/lib/python3.12/site-packages/benchpots/datasets/physionet_2012.py:109: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  X = X.groupby("RecordID").apply(apply_fu

(2557, 48, 37)
(640, 48, 37)
Temos 79.7% valores faltantes em train_X


2026-02-10 18:30:34 [INFO]: Epoch 001 - training loss (MAE): 1.1009, validation MSE: 0.5520
2026-02-10 18:30:47 [INFO]: Epoch 002 - training loss (MAE): 0.8623, validation MSE: 0.4785
2026-02-10 18:31:07 [INFO]: Epoch 003 - training loss (MAE): 0.7962, validation MSE: 0.4392
2026-02-10 18:31:13 [INFO]: Epoch 004 - training loss (MAE): 0.7528, validation MSE: 0.4130
2026-02-10 18:31:34 [INFO]: Epoch 005 - training loss (MAE): 0.7226, validation MSE: 0.3994
2026-02-10 18:31:36 [WARNING]: ‼️ Training got interrupted by the user. Exist now ...
2026-02-10 18:31:36 [INFO]: Finished training. The best model is from epoch#5.
2026-02-10 18:31:38 [INFO]: Successfully created the given path save_it_here
2026-02-10 18:31:38 [INFO]: Saved the model to save_it_here/brits_physionet2012.pypots
2026-02-10 18:31:38 [INFO]: Model loaded successfully from save_it_here/brits_physionet2012.pypots


MAE: 0.3091
